In [1]:
# -*- coding: utf-8 -*-
"""chi_squared_conf_int_1.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1FzaGtemF0sENWHADsdlAGhuplmYdGhIV
"""

import math

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from statistics import NormalDist


def get_intervals(probs: list[int]):
    intervals = []
    start = 0.0
    for prob in probs:
        intervals.append((start, start + prob))
        start += prob
    if not math.isclose(start, 1.0):
        raise ValueError("sum of probabilities must be equal to 1")
    return intervals


def random_counts(rng: np.random.Generator, probs: list[int], n: int):
    from string import ascii_uppercase

    if len(probs) > len(ascii_uppercase):
        raise ValueError("too many categories")
    labels = list(ascii_uppercase[: len(probs)])

    return pd.Series(
        pd.cut(
            rng.random((n,)),
            pd.IntervalIndex.from_tuples(
                get_intervals(probs),
                closed="left",
            ),
        ).rename_categories(labels)
    ).value_counts(sort=False)


def sampling_dist(sample_fn, trials=100):
    sample: pd.Series = sample_fn()
    n = sample.sum()
    freqs = {cat: list() for cat in sample.index}

    for _ in range(trials):
        sample = sample_fn()
        for cat, count in sample.items():
            freqs[cat].append(count)

    return pd.DataFrame(freqs) / n


def plot_df_hist(df: pd.DataFrame, *, bar_width=0.02):
    maxs = df.max()
    mins = df.min()

    fig, axs = plt.subplots(
        1,
        len(df.columns),
        figsize=(10, 5),
        sharey=True,
        tight_layout=True,
        width_ratios=maxs - mins,
    )

    for i, col in enumerate(df.columns):
        axs[i].hist(df[col], bins=np.arange(mins[col], maxs[col], bar_width))
        axs[i].set_title(col)
    fig.supxlabel("Sample Proportion")
    fig.supylabel("Counts")
    return fig


def plot_df_hist_overlay(df: pd.DataFrame, *, bar_width=0.02):
    df_max = df.max().max()
    df_min = df.min().min()
    bins = np.arange(df_min, df_max, bar_width)

    for col in df.columns:
        plt.hist(df[col], bins=bins, alpha=0.5, label=col)
    plt.legend()
    plt.xlabel("Sample Proportion")
    plt.ylabel("Counts")


def one_prop_z_int(props, n: int, conf: float = 0.95):
    norm = NormalDist()
    intervals = []
    z_critical = norm.inv_cdf((1 - conf) / 2)
    for prop in props:
        me = abs(z_critical * math.sqrt(prop * (1 - prop) / n))
        intervals.append((prop - me, prop + me))
    return intervals


def within_interval(
    props: pd.DataFrame, population_prop: list[float], n: int, conf: float = 0.95
):
    result = np.empty(props.shape, bool)
    for i, row in props.iterrows():
        intervals = one_prop_z_int(list(row), n=n, conf=conf)
        for j, (lo, hi) in enumerate(intervals):
            result[i][j] = lo < population_prop[j] < hi

    return pd.DataFrame(result, columns=props.columns)


In [2]:
from dataclasses import dataclass


@dataclass
class CategoricalPopulation:
    rng: np.random.Generator
    probs: list[int]
    n: int

    def one_sample(self):
        return random_counts(self.rng, self.probs, self.n)

    def sampling_dist(self, trials: int):
        return sampling_dist(self.one_sample, trials=trials)

    def within_interval_counts(
        self,
        conf: float,
        *,
        trials: int | None = None,
        dist: pd.DataFrame | None = None,
    ) -> pd.Series:
        if dist is None:
            if trials is None:
                raise ValueError("must provide either trials or dist")
            else:
                dist = self.sampling_dist(trials)

        return within_interval(
            props=dist, population_prop=self.probs, n=self.n, conf=conf
        ).sum(axis=1)

In [3]:
pop = CategoricalPopulation(
    np.random.default_rng(),
    [0.1, 0.2, 0.3, 0.4],
    200,
)
dist = pop.sampling_dist(5000)
pop.within_interval_counts(0.95, dist=dist).value_counts() / 5000

,count
4,0.8002
3,0.1612
2,0.0356
1,0.0028
0,0.0002


In [4]:
sample_sizes = [50, 100, 200, 500, 1000, 2000]
parameters = [
    [0.3, 0.7],
    [0.1, 0.3, 0.6],
    [0.2, 0.3, 0.5],
    [0.25, 0.35, 0.4],
    [1 / 3, 1 / 3, 1 / 3],
    [0.1, 0.2, 0.3, 0.4],
    [0.15, 0.25, 0.3, 0.3],
    [0.25, 0.25, 0.25, 0.25],
    [0.1, 0.1, 0.1, 0.1, 0.6],
    [0.15, 0.15, 0.15, 0.15, 0.4],
    [0.2, 0.2, 0.2, 0.2, 0.2],
]

import itertools

populations = []
rng = np.random.default_rng()
for n, probs in itertools.product(sample_sizes, parameters):
    if min(probs) * n < 10:
        # success/failure condition not met
        continue

    populations.append(CategoricalPopulation(rng, probs, n))

print(populations[0])


CategoricalPopulation(rng=Generator(PCG64) at 0x7E9E4C8B03C0, probs=[0.3, 0.7], n=50)


In [ ]:
for pop in populations:
    for _ in range(10):
        print(pop)
        print(pop.within_interval_counts(0.95, trials=5000).value_counts() / 5000)
        print()

CategoricalPopulation(rng=Generator(PCG64) at 0x7E9E4C8B03C0, probs=[0.3, 0.7], n=50)
2    0.938
0    0.062
Name: count, dtype: float64

CategoricalPopulation(rng=Generator(PCG64) at 0x7E9E4C8B03C0, probs=[0.3, 0.7], n=50)
2    0.9294
0    0.0706
Name: count, dtype: float64

CategoricalPopulation(rng=Generator(PCG64) at 0x7E9E4C8B03C0, probs=[0.3, 0.7], n=50)
2    0.935
0    0.065
Name: count, dtype: float64

CategoricalPopulation(rng=Generator(PCG64) at 0x7E9E4C8B03C0, probs=[0.3, 0.7], n=50)
2    0.9278
0    0.0722
Name: count, dtype: float64

CategoricalPopulation(rng=Generator(PCG64) at 0x7E9E4C8B03C0, probs=[0.3, 0.7], n=50)
2    0.9328
0    0.0672
Name: count, dtype: float64

CategoricalPopulation(rng=Generator(PCG64) at 0x7E9E4C8B03C0, probs=[0.3, 0.7], n=50)
2    0.9348
0    0.0652
Name: count, dtype: float64

CategoricalPopulation(rng=Generator(PCG64) at 0x7E9E4C8B03C0, probs=[0.3, 0.7], n=50)
2    0.9304
0    0.0696
Name: count, dtype: float64

CategoricalPopulation(rng=Gener